# 基于已有书名列表的豆瓣专题爬虫

- 需求：根据含有书名的Excel文件逐条查找补充书目信息，爬区豆瓣上的出版社、出版时间、ISBN、定价、评分、评分人数等；整合到pandas进行专题分析。
- 爬虫+数据分析：根据书名爬豆瓣获取书籍属性、整合到pandas，进行分析

本来打算直接用https://book.douban.com/subject_search?search_text={0}&cat=1001'.format('书名') 查询结构，没找到异步返回的数据json，绕了一圈后发现https://book.douban.com/j/subject_suggest?q={0}  可以用，就用了，通过比较名字和作者等确定url，然后直接爬该图书主页，然后写入字典。再变成dataframe。

In [1]:
#引入所需库
import json
import requests
import pandas as pd
from lxml import etree

In [4]:
bsdf=pd.read_excel('booklistfortest.xlsx') #读取书名Excel数据，为了简化，这里只用了"书名"列
#bsdf.head(3)

In [5]:
blst=list(bsdf['书名']) #书名列表
blst[0]

'龙族'

爬虫方面涉及到的：get，xpath去匹配，对于长度不定的一个html区域，不能写死xpath解析式，通过循环去取数，理清其HTML树结构；

一般解析是用BeautifulSoup或xpath，我更喜欢用xpath，因此下面的代码主要基于xpath解析文本。
以评分为例，鼠标点击评分部分，然后按Ctrl+Shift+I，或者右键点击检查元素，反正就是定位到评分对应的HTML上，定位到评分的代码部分后，右键，选择Copy->Copy XPath ,例如对于评分来说有：//*[@id="interest_sectl"]/div/div[2]/strong 通过con.xpath('//*[@id="interest_sectl"]/div/div[2]/strong/text()')就可以得到评分数据，返回的是列表，一般就是第0个值。同样，其他地方也是这样，而作者、出版社那几个属性是结构比较散的，需要特殊处理。

通过//*[@id="info"]/span[2] 可以确定 出版社 这个属性，但是属性的值，具体是哪个出版社不能确定，这些文字是在info这个节点上的。还是需要建立info的树结构

In [19]:
con.xpath('//*[@id="interest_sectl"]/div/div[2]/strong/text()')

[' 5.9 ']

In [8]:
def getBookInfo(binfo,cc):
    i=0
    rss={}
    k=''
    v=''
    f=0
    clw=[]
    for c in cc:
        if '\n' in c:
            if '\xa0' in c:
                clw.append(c)
        else:
            clw.append(c)
    
    for m in binfo[0]:
        if m.tag=='span':
            mlst=m.getchildren()
            if len(mlst)==0: 
                k=m.text.replace(':','')
                if '\xa0' in clw[i]:
                    f=1#需要m.tag=='a'下的值
                else:
                    v=clw[i].replace('\n','').replace(' ','')
                i+=1
            elif len(mlst)>0:#下面有子span 一种判断是m.attrib=={} 不够精确
                for n in mlst:
                    if n.tag=='span':
                        k=n.text.replace('\n','').replace(' ','') #不至于下面还有span，懒得用递归了
                    elif n.tag=='a':
                        v=n.text.replace('\n','').replace(' ','')
        
        elif m.tag=='a':
            if f==1: #是否可以不用这个if
                v=m.text.replace('\n','').replace(' ','')
                f=0
        elif m.tag=='br':
            if k=='':
                print(i,'err')
            else:
                rss[k]=v
        else:
            print(m.tag,i)
    return rss

调用上面函数返回的是一个字典，要整合到已有的字典里。涉及字典的组合，查了一下可以用d=dict(d,\*\*dw)，其中d是旧字典，dw是要加到d里的新字典，更简便的方式是用d.update(dw)函数，下面的代码就是用的update的。

主体代码如下，这个部分也可以打包为函数，因为jupyter里还是逐步分析方面就没打包

In [9]:
rlst=[]
for bn in blst:
    res={}
    r=requests.get('https://book.douban.com/j/subject_suggest?q={0}'.format(bn))
    rj=json.loads(r.text)
    #对rj进行一下验证和筛选
    html=requests.get(rj[0]['url']) #之后再考虑多个返回值的验证
    con = etree.HTML(html.text)
    bname=con.xpath('//*[@id="wrapper"]/h1/span/text()')[0] #和bn比较
    res['bname_sq']=bn
    res['bname']=bname
    res['dbid']=rj[0]['id'] #不需要存url，存id就够了
    #这部分取到info就够了，之后再用高级方法去匹配需要的元素，目前对应不对
    binfo=con.xpath('//*[@id="info"]')
    cc=con.xpath('//*[@id="info"]/text()')
    res.update(getBookInfo(binfo,cc))  #调用上面的函数处理binfo
    bmark=con.xpath('//*[@id="interest_sectl"]/div/div[2]/strong/text()')[0]
    if bmark=='  ':
        bits=con.xpath('//*[@id="interest_sectl"]/div/div[2]/div/div[2]/span/a/text()')[0]
        if bits=='评价人数不足':
            res['评分']=''
            res['评价人数']='评价人数不足'
        else:
            res['评分']=''
            res['评价人数']=''
    else:
        res['评分']=bmark.replace(' ','')
        bmnum=con.xpath('//*[@id="interest_sectl"]/div/div[2]/div/div[2]/span/a/span/text()')[0]
        res['评价人数']=bmnum
    rlst.append(res)

In [1]:
outdf=pd.DataFrame(rlst)

NameError: name 'pd' is not defined

In [2]:
outdf.head(3)

NameError: name 'outdf' is not defined

In [ ]:
#数据分析

In [ ]:
outdf.to_excel('out_douban_binfo.xlsx',index=False) #输出数据

## 试代码区域

In [18]:
r=requests.get('https://book.douban.com/j/subject_suggest?q={0}'.format('未来简史'))
rj=json.loads(r.text)
rj

[{'title': '未来简史',
  'url': 'https://book.douban.com/subject/26943161/',
  'pic': 'https://img3.doubanio.com/view/subject/s/public/s29287103.jpg',
  'author_name': '[以色列] 尤瓦尔·赫拉利',
  'year': '2017',
  'type': 'b',
  'id': '26943161'},
 {'title': '未来简史',
  'url': 'https://book.douban.com/subject/26945094/',
  'pic': 'https://img1.doubanio.com/view/subject/s/public/s29317658.jpg',
  'author_name': '[以色列] 尤瓦尔·赫拉利',
  'year': '2016',
  'type': 'b',
  'id': '26945094'},
 {'title': '未来简史',
  'url': 'https://book.douban.com/subject/4891197/',
  'pic': 'https://img3.doubanio.com/view/subject/s/public/s6263884.jpg',
  'author_name': '(法) 阿塔利',
  'year': '2010',
  'type': 'b',
  'id': '4891197'},
 {'title': '未來簡史',
  'url': 'https://book.douban.com/subject/30349873/',
  'pic': 'https://img3.doubanio.com/view/subject/s/public/s29893492.jpg',
  'author_name': '賈克‧阿塔利',
  'year': '2018',
  'type': 'b',
  'id': '30349873'},
 {'title': '未来人类简史',
  'url': 'https://book.douban.com/subject/26977428/',
 

返回的是json。
rj里面可以提取的属性：
- "title":"书名",
- "url":"对应书的网页",
- "pic":"书的封面图资源位置",
- "author_name":"作者",
- "year":"出版年份",
- "type":"类型，书籍一般标为b，应该是指book",
- "id":"书的豆瓣id，可以对应其网页"

eg:
- 'title': '未来简史',
- 'url': 'https://book.douban.com/subject/26943161/',
- 'pic': 'https://img3.doubanio.com/view/subject/s/public/s29287103.jpg',
- 'author_name': '[以色列] 尤瓦尔·赫拉利',
- 'year': '2017',
- 'type': 'b',
- 'id': '26943161'

In [20]:
#一般有多个返回值，简化只取第一个，所以就是用rj[0]['url']
html=requests.get(rj[0]['url'])

In [21]:
rj[0]['url']

'https://book.douban.com/subject/26943161/'

In [22]:
con = etree.HTML(html.text)
binfo=con.xpath('//*[@id="info"]')
binfo[0]

<Element div at 0x19aad51ed08>

In [24]:
getBookInfo(binfo,cc)

TypeError: getBookInfo() missing 1 required positional argument: 'cc'

In [ ]:
import re
def priceNorm(df):#用apply处理定价列
    pri=df['定价']
    pric=re.findall('^[0-9]+\.[0-9]+',pri)
    if pric==[]:
        print(pri)
        return float(pri)
    else:
        return float(pric[0])

In [ ]:
outdf.apply(priceNorm,axis=1)
#因为存在外国价格，就不直接转float了

## 参考文献

- https://lxml.de/api/lxml.etree._Element-class.html